从大规模数据集中寻找物品间的隐含关系被称作关联分析（association analysis）或者关联规则学习（association rule learning）。

## 1、关联分析

> 优点：易编码实现

> 缺点：在大数据集上可能较慢

> 适用数据类型：数值型或标称型

## 2、Apriori原理

Apriori原理的是说如果某个项集是频繁的，那么它的所有子集也是频繁的，反过来说，如果一个项集是非频繁集，那么它的所有超集也是非频繁的。

Apriori再拉丁语中指“来自以前”。当定义问题时，通常会使用先验知识或者假设，这被称作“一个先验”（a priori）。在贝叶斯统计中，使用先验知识作为条件进行推断也很常见。

## 3、使用Apriori算法来发现频繁集

In [1]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [2]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))

In [16]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = len(D)
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData

In [5]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [6]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [7]:
D = list(map(set, dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [23]:
L1, supportData = scanD(D, C1, 0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [24]:
supportData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

In [19]:
def aprioirGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

In [20]:
def apriori(dataSet, minSupport=0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioirGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [21]:
L, supportData = apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [22]:
supportData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

## 4、从频繁项集中挖掘关联规则

In [34]:
# 关联规则生成函数
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    # 只获取有两个或更多元素的集合
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

In [31]:
# 生成候选规则集合
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet-conseq]
        if conf >= minConf:
            print('{0}-->{1} conf: {2}'.format(freqSet-conseq, conseq, conf))
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [30]:
# 对规则评估
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m+1)):
        Hmp1 = aprioirGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [32]:
L, supportData = apriori(dataSet, minSupport=0.5)

In [36]:
rules = generateRules(L, supportData, minConf=0.7)
rules

frozenset({5})-->frozenset({2}) conf: 1.0
frozenset({2})-->frozenset({5}) conf: 1.0
frozenset({1})-->frozenset({3}) conf: 1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [37]:
rules = generateRules(L, supportData, minConf=0.5)
rules

frozenset({3})-->frozenset({2}) conf: 0.6666666666666666
frozenset({2})-->frozenset({3}) conf: 0.6666666666666666
frozenset({5})-->frozenset({3}) conf: 0.6666666666666666
frozenset({3})-->frozenset({5}) conf: 0.6666666666666666
frozenset({5})-->frozenset({2}) conf: 1.0
frozenset({2})-->frozenset({5}) conf: 1.0
frozenset({3})-->frozenset({1}) conf: 0.6666666666666666
frozenset({1})-->frozenset({3}) conf: 1.0
frozenset({5})-->frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3})-->frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2})-->frozenset({3, 5}) conf: 0.6666666666666666


[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

## 5、示例：发现毒蘑菇的相似特征

In [39]:
mushDatSet = [line.split() for line in open('mushroom.dat').readlines()]

In [44]:
L, supportData = apriori(mushDatSet, minSupport=0.3)

In [46]:
# 可以在结果中搜索包含有毒特征值2的频繁项集
for item in L[1]:
    if item.intersection('2'):
        print(item)

frozenset({'28', '2'})
frozenset({'53', '2'})
frozenset({'23', '2'})
frozenset({'34', '2'})
frozenset({'36', '2'})
frozenset({'59', '2'})
frozenset({'63', '2'})
frozenset({'67', '2'})
frozenset({'76', '2'})
frozenset({'85', '2'})
frozenset({'86', '2'})
frozenset({'90', '2'})
frozenset({'93', '2'})
frozenset({'39', '2'})


In [47]:
for item in L[3]:
    if item.intersection('2'):
        print(item)

frozenset({'59', '34', '28', '2'})
frozenset({'63', '34', '28', '2'})
frozenset({'63', '59', '28', '2'})
frozenset({'85', '53', '28', '2'})
frozenset({'85', '39', '28', '2'})
frozenset({'85', '34', '28', '2'})
frozenset({'85', '59', '28', '2'})
frozenset({'85', '63', '28', '2'})
frozenset({'85', '86', '28', '2'})
frozenset({'85', '90', '28', '2'})
frozenset({'86', '34', '28', '2'})
frozenset({'86', '59', '28', '2'})
frozenset({'86', '63', '28', '2'})
frozenset({'86', '53', '28', '2'})
frozenset({'90', '34', '28', '2'})
frozenset({'90', '59', '28', '2'})
frozenset({'90', '86', '28', '2'})
frozenset({'90', '53', '28', '2'})
frozenset({'53', '34', '28', '2'})
frozenset({'39', '53', '28', '2'})
frozenset({'39', '34', '28', '2'})
frozenset({'39', '59', '28', '2'})
frozenset({'39', '63', '28', '2'})
frozenset({'39', '86', '28', '2'})
frozenset({'39', '90', '28', '2'})
frozenset({'85', '39', '53', '2'})
frozenset({'85', '34', '53', '2'})
frozenset({'85', '86', '53', '2'})
frozenset({'85', '90

> 总结：关联分析适用于发现大数据集中元素见有趣关系的一个工具集，可以才两种方式来量化，第一种是使用频繁项集，第二种方式是关联规则。

> Apriori算法使用Apriori原理来提高计算的效率。